# Generación de Melodías

Este notebook está dedicado a programar el código necesario para llevar a cabo el proyecto apoyándose en las herramientas del otro notebook. Para esto, se crea una clase llamada MelodyGenerator, que almacena un modelo de Markov (a elegir entre una cadena simple y un HMM) y métodos para recolectar datos de entrenamiento, entrenar dicho modelo y utilizar el resultado para generar melodías.

*Por:*
- Sebastián Toloza
- Benjamín Valdés Vera

Bloque de instalación e importación:

In [1]:
import os
import mido
from algoritmos import *

## Clase MelodyGenerator

Esta clase es una interface con las dos clases detalladas en AlgoritmosHMM. Se encarga del procesamiento MIDI y de llamar apropiadamente a los métodos de ambas clases para generar una melodía según data de entrenamiento previamente seleccionada de tracks MIDI.

In [25]:
class MelodyGenerator:
    """MelodyGenerator: Clase que intermedia entre el estandar MIDI y los modelos prrogramados.
    Puede contener un HMM o un OMM a especificar."""
    def __init__(self, model_type, hidden_states):
        """Recibe el tipo de modelo deseado y la cantidad de estados ocultos (solo usada en HMM).
        Crea una instancia MelodyGenerator sin data de entenamiento aún"""
        assert model_type in ["HMM", "OMM"], f"El tipo del modelo dado debe ser HMM o bien OMM, recibí {model_type}"
        self.model_type = model_type
        self.hidden_states = hidden_states
        self.model = None
        self.training_data = None

        # Unidades de traducción:
        Notas = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
        self.midi_to_note = {n: Notas[n%12] + str((n//12) - 1) for n in range(128)}
        print(f"Modelo de tipo {self.model_type} creado exitosamente.")
    
    def load_training_data(self, folder="DPPt"):
        """Corre un bucle de interacción con el usuario para especificar las melodías a ser usadas
        durante el entrenamiento del modelo. Al final, crea el modelo al haber definido la cantidad
        de osbservables."""
        # Bucle de las melodías de entrenamiento
        L = []
        ready = False
        while not ready:
            # Bucle para pedir nombre válido de una canción
            name_is_valid = False
            while not name_is_valid:
                name = input("Ingrese nombre del archivo MIDI (ejemplo 'Item.mid'):\n")
                name_is_valid = os.path.join(name) in os.listdir(folder)
                if not name_is_valid:
                    print("Archivo no encontrado.")
            mid = mido.MidiFile(f"{folder}/{name}", clip=True)
            num_tracks = len(mid.tracks)

            # Bucle para pedir número válido de track mostrando previews
            user_has_decided = False
            while not user_has_decided:
                track_is_valid = False
                while not track_is_valid:
                    idx = int(input(f"El archivo seleccionado posee {num_tracks} tracks. Elija el índice del que quiere extraer:\n"))
                    track_is_valid = idx in range(num_tracks)
                    if not track_is_valid:
                        print("Track inválido")
                track = mid.tracks[idx]
                sample = [self.midi_to_note[m.note] for m in track if (m.type == 'note_on' and m.velocity != 0)]
                print(sample[:10])
                user_has_decided = input("Estas son las primeras 10 notas del track seleccionado ¿Mantiene su selección? [y/n]\n") == 'y'

            # Bucle para pedir transposición
            transpose_is_valid = False
            while not transpose_is_valid:
                transpose = input("¿Desea transponer la melodía extraída? Ingrese un número de semitonos\npor tranponer hacia arriba\nSi no desea transponer, ingrese 0:\n")
                transpose_is_valid = transpose.isnumeric()
            transpose = int(transpose)

            # Aplicamos transposición
            for m in track:
                if m.type == 'note_on':
                    m.note += transpose
            
            L += track
            ready = input("¿Desea continuar añadiendo melodías a la data de entrenamiento? [y/n]\n") == 'n'

        # Incorporamos una lista con los valores MIDI de las notas
        # PENDIENTE: En el estado actual nos estamos olvidando de los ritmos
        Notas_midi = []
        for m in L:
            if m.type == 'note_on' and m.velocity != 0:
                Notas_midi.append(m.note)
        
        Notas_u = []
        for n in Notas_midi:
            if n not in Notas_u:
                Notas_u.append(n)
        Notas_u.sort()
        
        self.index_to_midi = {j: Notas_u[j] for j in range(len(Notas_u))}
        self.midi_to_index = {v: k for k, v in self.index_to_midi.items()}
        Notas_idx = [self.midi_to_index[nota] for nota in Notas_midi]
        self.training_data = np.array(Notas_idx)

        # Creamos el modelo
        if self.model_type == "HMM":  # Si es HMM, uso los estados ocultos y las notas como observables.
            N = self.hidden_states
            M = len(Notas_u)
            self.model = HMM(N, M)
        else:  
            N = len(Notas_u)
            self.model = OMM(N)

        print("Proceso de carga de data de entrenamiento finalizado.")

    def train(self, verbose=False):
        """Llama al método de entrenamiento de su correspondiente modeo, pidiendo información
        adicional si es necesario."""
        assert self.model is not None, "Cargue data de entrenamiento antes de entrenar."
        if self.model.__class__.__name__ == "HMM":
            eps = float(input("Entrenando un HMM. Ingrese el grado de tolerancia para medir la convergencia:\n"))
            num = int(input("Ingrese el número máximo de iteraciones:\n"))
            self.model.train(self.training_data, eps, num, verbose)
        elif self.model.__class__.__name__ == "OMM":
            # Si el modelo es OMM, utilizamos el método train con la secuencia de melodías
            self.model.train(self.training_data)
        print("Entrenamiento del modelo finalizado con éxito.")

    def generate(self, steps, return_type="MidiFile"):
        """Llama al método de simulación de su correspondiente modelo. Retorna una secuencia de
        notas en formato de string"""
        assert return_type in ["MidiFile", "Sequence"], f"return_type debe ser MidiFile o Sequence. Recibí {return_type}"
        seq_idx = self.model.simulate(steps)
        seq_midi = [self.index_to_midi[idx] for idx in seq_idx]
        if return_type == "Sequence":
            seq_notes = [self.midi_to_note[mid] for mid in seq_midi]
            print("Secuencia de notas generada exitosamente.")
            return seq_notes
        
        # Creo el archivo MIDI con un track
        MF = mido.MidiFile()
        MF.add_track()

        # Preámbulo
        Preamble = [
            mido.MetaMessage('track_name', name='Piano\x00', time=0),
            mido.MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
            mido.MetaMessage('key_signature', key='C', time=0),
            mido.MetaMessage('set_tempo', tempo=500000, time=0),
            mido.Message('control_change', channel=0, control=121, value=0, time=0),
            mido.Message('program_change', channel=0, program=0, time=0),
            mido.Message('control_change', channel=0, control=7, value=100, time=0),
            mido.Message('control_change', channel=0, control=10, value=64, time=0),
            mido.Message('control_change', channel=0, control=91, value=0, time=0),
            mido.Message('control_change', channel=0, control=93, value=0, time=0),
            mido.MetaMessage('midi_port', port=0, time=0),
        ]
        MF.tracks[0] += Preamble

        # Añadimos notas
        msg_on = mido.Message('note_on', channel=0, note=seq_midi[0], velocity=80, time=0)
        msg_off = mido.Message('note_off', channel=0, note=seq_midi[0], velocity=80, time=455)
        MF.tracks[0].append(msg_on)
        MF.tracks[0].append(msg_off)
        for m in seq_midi[1:]:
            msg_on = mido.Message('note_on', channel=0, note=m, velocity=80, time=25)
            msg_off = mido.Message('note_off', channel=0, note=m, velocity=80, time=455)
            MF.tracks[0].append(msg_on)
            MF.tracks[0].append(msg_off)
        # Finalizamos track
        MF.tracks[0].append(mido.MetaMessage('end_of_track', time=1))
        print("Archivo MIDI generado exitosamente.")
        return MF

## Experimentos

Aquí, experimentamos con el código programado para generar algunas melodías.

### 1. Paseo aleatorio por la escala mayor

Se utilizó la siguiente data de entrenamiento que corresponde a una escala de Do Mayor:

![Picture title](img/1train.png)

Con esto, se entrenó un modelo observable. La matriz de transición obtenida, por lo tanto, debería ser la de un paseo aleatorio simple. A continuación está el código para esto:

In [3]:
# Si es HMM, uso los estados ocultos y las notas como observables.
# Si es OMM, las notas mismas son los estados 
MG = MelodyGenerator("OMM", None) 
MG.load_training_data(folder="Experimentos") #cambiar el folder por la data para entrenar el algoritmo
MG.train()
mid = MG.generate(49) #se puede cambiar el numero de pasos
mid.save("Escala_generado.mid") #se retorna

Modelo de tipo OMM creado exitosamente.
['C5', 'D5', 'E5', 'F5', 'G5', 'A5', 'B5', 'C6', 'B5', 'A5']
Proceso de carga de data de entrenamiento finalizado.
Entrenamiento del modelo finalizado con éxito.
Archivo MIDI generado exitosamente.


Tras correr este código, este es un ejemplo de melodía que se obtiene:

![Picture title](img/1gen.png)

Notar que efectivamente el comportamiento es el de un paseo aleatorio simple. Inspeccionando manualmente la matriz de transición, se puede confirmar esto.

### 2. Route 201

Se utilizan dos instrumentos del archivo Route 201.mid: la parte de flauta y la de piano:

![Picture title](img/2train.png)

Nótese que como efecto de sonido, el archivo tiene muchas notas repetidas en ambas partes. Esto es a propósito para el OST del juego, pero podría generar una melodías que repita notas de manera poco armoniosa.

Se entrena un modelo de tipo HMM con 10 estados ocultos, cantidad elegida arbitrariamente.

In [12]:
# Si es HMM, uso los estados ocultos y las notas como observables.
# Si es OMM, las notas mismas son los estados 
MG = MelodyGenerator("HMM", 10) #se pueden cambiar la cantidad de estados ocultos
MG.load_training_data()
MG.train(verbose=True) #false para no printear informacion extra
mid = MG.generate(49) #se pueden cambiar el numero de pasos
mid.save("201HMM_piano.mid")

Modelo de tipo HMM creado exitosamente.
['E5', 'D5', 'C5', 'D5', 'C5', 'G4', 'E4', 'F4', 'E4', 'F4']
Proceso de carga de data de entrenamiento finalizado.
Iteración 1 | dA = 0.12 | dB = 0.68 | dlam = 0.24
Iteración 2 | dA = 0.05 | dB = 0.08 | dlam = 0.15
Iteración 3 | dA = 0.07 | dB = 0.11 | dlam = 0.15
Iteración 4 | dA = 0.13 | dB = 0.16 | dlam = 0.12
Iteración 5 | dA = 0.27 | dB = 0.27 | dlam = 0.10
Iteración 6 | dA = 0.31 | dB = 0.34 | dlam = 0.12
Iteración 7 | dA = 0.29 | dB = 0.28 | dlam = 0.13
Iteración 8 | dA = 0.27 | dB = 0.29 | dlam = 0.13
Iteración 9 | dA = 0.20 | dB = 0.21 | dlam = 0.07
Iteración 10 | dA = 0.23 | dB = 0.19 | dlam = 0.01
Iteración 11 | dA = 0.24 | dB = 0.19 | dlam = 0.00
Iteración 12 | dA = 0.21 | dB = 0.15 | dlam = 0.00
Iteración 13 | dA = 0.19 | dB = 0.14 | dlam = 0.00
Iteración 14 | dA = 0.19 | dB = 0.17 | dlam = 0.00
Iteración 15 | dA = 0.20 | dB = 0.17 | dlam = 0.00
Iteración 16 | dA = 0.19 | dB = 0.15 | dlam = 0.00
Iteración 17 | dA = 0.18 | dB = 0.14 |

### 3. Oreburgh City

Se utiliza solo la melodía central de la canción:

![Picture title](img/3train.png)

Con esto, se entrena un HMM con 15 estados ocultos:

In [27]:
# Si es HMM, uso los estados ocultos y las notas como observables.
# Si es OMM, las notas mismas son los estados 
MG = MelodyGenerator("HMM", 15) #se pueden cambiar la cantidad de estados ocultos
MG.load_training_data(folder="Experimentos")
MG.train(verbose=True)
mid = MG.generate(49) #se pueden cambiar el numero de pasos
mid.save("Oreburgh_15.mid")

Modelo de tipo OMM creado exitosamente.
['C5', 'G5', 'D5', 'E5', 'F5', 'G5', 'C6', 'A5', 'G5', 'C5']
Proceso de carga de data de entrenamiento finalizado.
Entrenamiento del modelo finalizado con éxito.
Archivo MIDI generado exitosamente.


Melodía generada:

![Picture title](img/3gen.png)

### 4. Bach

blablabla 

![Picture title](img/4train.png)

blablabla 15 estados ocultos.


In [24]:
# Si es HMM, uso los estados ocultos y las notas como observables.
# Si es OMM, las notas mismas son los estados 
MG = MelodyGenerator("OMM", 5) #se pueden cambiar la cantidad de estados ocultos
MG.load_training_data(folder="Experimentos")
MG.train(verbose=True)
mid = MG.generate(100) #se pueden cambiar el numero de pasos
mid.save("Bach_generado.mid")

Modelo de tipo OMM creado exitosamente.
['E6', 'D#6', 'E6', 'B5', 'G#5', 'B5', 'E5', 'F#5', 'E5', 'D#5']
Proceso de carga de data de entrenamiento finalizado.
Entrenamiento del modelo finalizado con éxito.
Archivo MIDI generado exitosamente.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=15d2711e-8488-4966-b405-4363c7f8973c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>